In [36]:
%cd ..
from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain_community.vectorstores import FAISS
import faiss
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import json
import re
from random import seed,shuffle
from time import sleep

/home/dp/hdd


In [2]:
import pandas as pd


In [3]:
from ragchat.doc_store import DocStore
from ragchat.configs import (DEBUG, REFERENCE_FOLDER, DB_NAME, QUESTIONS_VECTOR_STORE_SAVE_PATH,
COLLECTION_NAME, Q_DB_NAME, Q_COLLECTION_NAME, TOKENS_PER_SYNTH_QUESTION)
from ragchat.text_embedder import TextEmbedder


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


In [8]:
vs = FAISS.load_local('vector_stores/faiss_index_0c7394a1b7ff35a035b5adbc4dbb8fae',OpenAIEmbeddings(model="text-embedding-ada-002"))


In [13]:
vs.__dict__.keys()

dict_keys(['embedding_function', 'index', 'docstore', 'index_to_docstore_id', 'distance_strategy', 'override_relevance_score_fn', '_normalize_L2'])

In [40]:
num_docs = vs.index.ntotal
# Get the dimension of your embeddings
embedding_dimension = vs.index.d

embedding_arr = faiss.rev_swig_ptr(vs.index.get_xb(), num_docs*embedding_dimension).reshape(num_docs, embedding_dimension)
embedding_arr.shape

(14712, 1536)

In [15]:
vs.docstore.__dict__

{'_dict': {'e48f965e-ff0c-425e-b03e-f59f2cfb693f': Document(page_content='Collection_Name: Weekly Reports\nTitle: Nat Guard List Film and TV Shows.pdf\n\n\nATTACHMENT 1 and 2 \n\n \n\nList of films and TV shows that the National Guard has been involved with, taken from available records: \n\n  \n\n115th FW documentary Alabama National Cemetery video American Giving Awards American Ride Apache War Machine Australian Special Air Service Regiment, The - the Untold History Aviators, The AWOL Baggage Battles Battle LA Bomb Hunters Building Wild Cat Run 2 Celebrity Sweat Change of Heart Channel Islands ANG mission Chew, The Citizen Soldiers Colorado motorcycle TV project Combat Rescue Contagion Country Music project Crisis D-Day documentary Daily Planet Dirty Bomb Discovery Channel -- Tunnel DJ Dollz video Drones documentary Echo 360 Engineering Alaska Enlisted Espionage Everybody Loves Whales Eyewitness War Fantastic Four 2 Fight for Freedom Fix It Finish It Flag documentary Fort Bliss docu

In [12]:
vs.index_to_docstore_id

{0: 'e48f965e-ff0c-425e-b03e-f59f2cfb693f',
 1: 'bd33ef3e-6848-4a6b-994e-b0b2462b95b8',
 2: '00222636-c7da-4061-a019-4f05555c954e',
 3: '7360ea9b-a671-485e-8571-01d216bebba9',
 4: 'c280cf1e-8012-4ff8-9179-78b4c178899d',
 5: 'd5fceb82-ae18-4950-9003-e83391b4c8cc',
 6: '2a423b95-29e0-4970-a4b3-6df355025bbc',
 7: '745718b9-eda4-4f5d-b240-af6ab9ed4395',
 8: '4c98c511-6e8e-455f-a2cc-5ba62dbbd121',
 9: 'c321e3b7-f925-42c8-954a-556a0d1c8bd2',
 10: '7e358375-ca98-4d4d-92e5-c5cc0bbaa2c4',
 11: '2910d646-caa3-44f2-94d4-c82c4e99ba9b',
 12: 'fbd64984-3893-4ea4-9759-47bd776bc29a',
 13: 'bfaa20a3-f3f2-4395-a352-0919e0207cea',
 14: '2fecc3cd-69d3-4743-9cba-73ee5d0a2c38',
 15: '55a1aae7-99f2-46bc-a8c6-0c441fd4d161',
 16: 'e4cc138f-92bb-4957-9272-87cea26d605e',
 17: 'd4f67b74-2584-406a-b3d9-086c134a5247',
 18: '90848ede-039d-44b1-8f94-204934939099',
 19: '3458a439-5c39-4ba1-bd02-08ce520f8ebc',
 20: '67b7bb9f-1d4f-472f-abce-0bd6328a7022',
 21: '64838ea9-3100-43a7-babb-26d07e938949',
 22: '6902d63b-1e8f-